In [27]:
# aruco_dataset_builder_splitter.py

import os
import cv2
import csv
import json
import numpy as np
from tqdm import tqdm
from datetime import datetime
from aruco_box_pose_estimation import CubePoseEstimator
from utility.stereo_utils import Split_Stereo_Frame


In [28]:
# Configuration
BATCH_ID = 2
BATCH_FOLDER = f"dataset/batch{BATCH_ID}"
DATASET_VIDEO = "video/raw/ArUcoCom.avi"
UNSPLIT_FOLDER = os.path.join(BATCH_FOLDER, "unsplit")
IMAGES_FOLDER = os.path.join(UNSPLIT_FOLDER, "images")
LABELS_PATH = os.path.join(UNSPLIT_FOLDER, "labels.csv")
os.makedirs(IMAGES_FOLDER, exist_ok=True)

estimator = CubePoseEstimator()
cap = cv2.VideoCapture(DATASET_VIDEO)

In [29]:
with open(LABELS_PATH, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['image_name', 'x', 'y', 'z', 'pitch', 'roll', 'yaw'])

    frame_idx = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video reached or failed to read frame.")
                break

            cube_rvec, cube_tvec = estimator.process_frame(frame)
            if cube_rvec is not None:
                rmat, _ = cv2.Rodrigues(cube_rvec)
                pitch = np.degrees(np.arcsin(-rmat[2, 0]))
                roll = np.degrees(np.arctan2(rmat[2, 1], rmat[2, 2]))
                yaw = np.degrees(np.arctan2(rmat[1, 0], rmat[0, 0]))

                # Save image
                fname = f"frame_{frame_idx:04d}.png"
                image_path = os.path.join(IMAGES_FOLDER, fname)
                cv2.imwrite(image_path, frame)

                # Write label
                writer.writerow([
                    fname,
                    float(cube_tvec[0][0]),
                    float(cube_tvec[1][0]),
                    float(cube_tvec[2][0]),
                    pitch,
                    roll,
                    yaw
                ])

                print(f"Wrote frame {frame_idx:04d} and its label.")
                frame_idx += 1
    finally:
        cap.release()
        print("Video capture released.")

Wrote frame 0000 and its label.
Wrote frame 0001 and its label.
Wrote frame 0002 and its label.
Wrote frame 0003 and its label.
Wrote frame 0004 and its label.
Wrote frame 0005 and its label.
Wrote frame 0006 and its label.
Wrote frame 0007 and its label.
Wrote frame 0008 and its label.
Wrote frame 0009 and its label.
Wrote frame 0010 and its label.
Wrote frame 0011 and its label.
Wrote frame 0012 and its label.
Wrote frame 0013 and its label.
Wrote frame 0014 and its label.
Wrote frame 0015 and its label.
Wrote frame 0016 and its label.
Wrote frame 0017 and its label.
Wrote frame 0018 and its label.
Wrote frame 0019 and its label.
Wrote frame 0020 and its label.
Wrote frame 0021 and its label.
Wrote frame 0022 and its label.
Wrote frame 0023 and its label.
Wrote frame 0024 and its label.
Wrote frame 0025 and its label.
Wrote frame 0026 and its label.
Wrote frame 0027 and its label.
Wrote frame 0028 and its label.
Wrote frame 0029 and its label.
Wrote frame 0030 and its label.
Wrote fr

DATASET SPLIT

In [30]:
import shutil
import random

In [31]:
# Load labels
with open(LABELS_PATH, 'r') as f:
    reader = list(csv.reader(f))
    header = reader[0]
    rows = reader[1:]

In [32]:
trRatio, tsRatio, vlRatio = 0.7,0.3,0
                #70 - 30 SPLIT
# trRatio, tsRatio, vlRatio = 0.8,0.2,0
                #80 - 20 SPLIT
# trRatio, tsRatio, vlRatio = 0.6,0.4,0
                #60 - 40 SPLIT
# trRatio, tsRatio, vlRatio = 0.7,0.2,0.1
                #70 - 20 - 10 SPLIT

In [33]:
random.seed(42)
random.shuffle(rows)

total = len(rows)
train_end = int(trRatio * total)
val_end = train_end + int(vlRatio * total)

train = rows[:train_end]
val = rows[train_end:val_end]
test = rows[val_end:]

In [34]:
# Create subfolders
for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(BATCH_FOLDER, subset, 'images'), exist_ok=True)

In [35]:
# Write splits and copy images
for subset_name, data in zip(['train', 'val', 'test'], [train, val, test]):
    csv_path = os.path.join(BATCH_FOLDER, subset_name, 'labels.csv')
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)
            src_img = os.path.join(IMAGES_FOLDER, row[0])
            dst_img = os.path.join(BATCH_FOLDER, subset_name, 'images', row[0])
            shutil.copy(src_img, dst_img)

In [36]:
# Write metadata.md
frame = cv2.imread(f"dataset/batch{BATCH_ID}/unsplit/images/frame_0001.png")

metadata_path = os.path.join(BATCH_FOLDER, 'metadata.md')
with open(metadata_path, 'w') as f:
    f.write(f"# Batch {BATCH_ID} Metadata\n")
    f.write(f"- Date: {datetime.now().isoformat()}\n")
    f.write(f"- Total Frames: {len(rows)}\n")
    f.write(f"- Train: {len(train)} | Val: {len(val)} | Test: {len(test)}\n")
    f.write(f"- Image Resolution: {frame.shape[1]}x{frame.shape[0]}\n")
    f.write(f"- ArUco-based 6DoF Pose Estimation\n")
    f.write(f"- Cube Size: {estimator.CUBE_SIZE if hasattr(estimator, 'CUBE_SIZE') else 0.08}m\n")

print(f"Metadata written to {metadata_path}")

Metadata written to dataset/batch2\metadata.md
